# Bookmarks
Preprocessing [#1](#pp1)

# Start of Notebook

In [ ]:
!pip uninstall -y pandas numpy

Found existing installation: pandas 1.3.5
Uninstalling pandas-1.3.5:
  Would remove:
    /home/cdsw/.local/lib/python3.7/site-packages/pandas-1.3.5.dist-info/*
    /home/cdsw/.local/lib/python3.7/site-packages/pandas/*
Proceed (y/n)? 

In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql import functions as F

## Starting Spark Session

Function to set environment variable STORAGE

In [1]:
import os
import xml.etree.ElementTree as ET

# Set the STORAGE environment variable (Thanks to Paul DeFusco)
try:
    storage = os.environ["STORAGE"]
except:
    if os.path.exists("/etc/hadoop/conf/hive-site.xml"):
        tree = ET.parse("/etc/hadoop/conf/hive-site.xml")
        root = tree.getroot()
        for prop in root.findall("property"):
            if prop.find("name").text == "hive.metastore.warehouse.dir":
                storage = (
                    prop.find("value").text.split("/")[0]
                    + "//"
                    + prop.find("value").text.split("/")[2]
                )
    else:
        storage = "/user/" + os.getenv("HADOOP_USER_NAME")
  #  storage_environment_params = {"STORAGE": storage}
  #  storage_environment = cml.create_environment_variable(storage_environment_params)
    os.environ["STORAGE"] = storage

In [ ]:
# spark = SparkSession\
#     .builder\
#     .appName("Amex Analysis")\
#     .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
#     .config("spark.yarn.access.hadoopFileSystems",os.environ["STORAGE"])\
#     .getOrCreate()

In [ ]:
# df_train = spark.read.option('inferschema','true').csv(
#   os.environ["STORAGE"] + "/ps-ml/american-express-kaggle/data/train_data.csv",
#   header=True,
#   sep=','
# )

Loading dataset

In [6]:
import pandas as pd

In [7]:
df_train = pd.read_csv(
    "/home/cdsw/data/train_data.csv",
    engine='c'
    #"s3://ps-uat2/ps-ml/american-express-kaggle/data/train_data.csv"
)

KeyboardInterrupt: 

**Categorical Features provided: 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'**

In [ ]:
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [ ]:
# df_labels = spark.read.option('inferschema','true').csv(
#     os.environ["STORAGE"] + "/ps-ml/american-express-kaggle/data/train_labels.csv",
#     header=True,
#     sep=','
# )

In [ ]:
# df_labels.head(1)

In [ ]:
# train_rec_count = df_train.count()
# train_rec_count

## Checking null values count

In [ ]:
null_pct = df_train.select([(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c))/train_rec_count).alias(c) for c in df_train.columns]).toPandas()
null_pct

In [ ]:
null_25pct_cols = null_pct.T[(null_pct.T > 0.25).any(axis=1)].index
null_25pct_cols

In [ ]:
len(null_1pct_cols)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
null_pct[null_25pct_cols]

# Preprocessing #1 <a name='pp1'/>
Dropping columns with more than 25% null values

In [ ]:
pd_df_train = df_train_nonnull.toPandas()

## Filling null values with mean value

List columns with null values

In [ ]:
len(null_pct.T[((null_pct.T < 0.01) & (null_pct.T > 0.0)).any(axis=1)].index)